In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import datawig as dw
import mxnet as mx
import optuna
import re
import gc
import time

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.datasets import make_moons
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer



from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering3 import FeatureEngineering3
from FeatureEngineering5 import FeatureEngineering5


%matplotlib inline

print("OK\n")

OK



In [2]:
baseline = Baseline('TARGET')
modules = Modules('TARGET')
FeatureEngineering2 = FeatureEngineering2('TARGET')
FeatureEngineering3 = FeatureEngineering3('TARGET')
FeatureEngineering5 = FeatureEngineering5('TARGET')

In [3]:
"""
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}

def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

In [4]:
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 2000
}

In [5]:
#with FeatureEngineering5.timer("Pipeline total time"):
#    FeatureEngineering5.main(debug= False)

In [6]:
num_rows = 30000

with FeatureEngineering5.timer("application_train and application_test"):
    df = FeatureEngineering5.get_train_test(FeatureEngineering5.DATA_DIRECTORY, num_rows= num_rows)
    print("Application dataframe shape: ", df.shape)
with FeatureEngineering5.timer("Bureau and bureau_balance data"):
    bureau_df = FeatureEngineering5.get_bureau(FeatureEngineering5.DATA_DIRECTORY, num_rows= num_rows)
    df = pd.merge(df, bureau_df, on='SK_ID_CURR', how='left')
    print("Bureau dataframe shape: ", bureau_df.shape)
    del bureau_df; gc.collect()
with FeatureEngineering5.timer("previous_application"):
    prev_df = FeatureEngineering5.get_previous_applications(FeatureEngineering5.DATA_DIRECTORY, num_rows)
    df = pd.merge(df, prev_df, on='SK_ID_CURR', how='left')
    print("Previous dataframe shape: ", prev_df.shape)
    del prev_df; gc.collect()
with FeatureEngineering5.timer("previous applications balances"):
    pos = FeatureEngineering5.get_pos_cash(FeatureEngineering5.DATA_DIRECTORY, num_rows)
    df = pd.merge(df, pos, on='SK_ID_CURR', how='left')
    print("Pos-cash dataframe shape: ", pos.shape)
    del pos; gc.collect()
    ins = FeatureEngineering5.get_installment_payments(FeatureEngineering5.DATA_DIRECTORY, num_rows)
    df = pd.merge(df, ins, on='SK_ID_CURR', how='left')
    print("Installments dataframe shape: ", ins.shape)
    del ins; gc.collect()
    cc = FeatureEngineering5.get_credit_card(FeatureEngineering5.DATA_DIRECTORY, num_rows)
    df = pd.merge(df, cc, on='SK_ID_CURR', how='left')
    print("Credit card dataframe shape: ", cc.shape)
    del cc; gc.collect()
# Add ratios and groupby between different tables
df = FeatureEngineering5.add_ratios_features(df)
df.head()

Application dataframe shape:  (356250, 83)
application_train and application_test - done in 24s
Bureau dataframe shape:  (305811, 156)
Bureau and bureau_balance data - done in 21s
Previous dataframe shape:  (338857, 225)
previous_application - done in 27s
Pos-cash dataframe shape:  (337252, 27)
Installments dataframe shape:  (339587, 101)
Credit card dataframe shape:  (103558, 59)
previous applications balances - done in 1112s


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,CURRENT_TO_APPROVED_ANNUITY_MEAN_RATIO,PAYMENT_MIN_TO_ANNUITY_RATIO,PAYMENT_MAX_TO_ANNUITY_RATIO,PAYMENT_MEAN_TO_ANNUITY_RATIO,CTA_CREDIT_TO_ANNUITY_MAX_RATIO,CTA_CREDIT_TO_ANNUITY_MEAN_RATIO,DAYS_DECISION_MEAN_TO_BIRTH,DAYS_CREDIT_MEAN_TO_BIRTH,DAYS_DECISION_MEAN_TO_EMPLOYED,DAYS_CREDIT_MEAN_TO_EMPLOYED
0,100002,1.0,0,0,0,202500.0,406597.5,24700.5,351000.0,0,...,0.374558,0.374558,2.149501,0.467976,1.175716,1.175716,0.064052,0.092379,0.951334,1.372057
1,100003,0.0,0,1,0,270000.0,1293502.5,35698.5,1129500.0,1,...,1.584212,0.186646,15.710334,1.813930,0.290662,0.239484,0.077841,0.083552,1.098485,1.179082
2,100004,0.0,1,0,1,67500.0,135000.0,6750.0,135000.0,0,...,0.793667,0.793667,1.566513,1.051282,0.187652,0.187652,0.042791,0.045521,3.622222,3.853333
3,100006,0.0,0,1,0,135000.0,312682.5,29686.5,297000.0,0,...,0.735762,0.083638,23.303080,2.120394,2.643133,1.497703,0.018185,NaN,0.113722,NaN
4,100007,0.0,0,0,0,121500.0,513000.0,21865.5,513000.0,0,...,0.561561,0.000008,1.037195,0.558600,0.931669,0.538926,0.061350,0.057646,0.402513,0.378209


In [7]:
df = FeatureEngineering5.reduce_memory(df)
df.head()

Initial df memory usage is 1802.02 MB for 662 columns
Final memory usage is: 702.94 MB - decreased by 61.0%


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,CURRENT_TO_APPROVED_ANNUITY_MEAN_RATIO,PAYMENT_MIN_TO_ANNUITY_RATIO,PAYMENT_MAX_TO_ANNUITY_RATIO,PAYMENT_MEAN_TO_ANNUITY_RATIO,CTA_CREDIT_TO_ANNUITY_MAX_RATIO,CTA_CREDIT_TO_ANNUITY_MEAN_RATIO,DAYS_DECISION_MEAN_TO_BIRTH,DAYS_CREDIT_MEAN_TO_BIRTH,DAYS_DECISION_MEAN_TO_EMPLOYED,DAYS_CREDIT_MEAN_TO_EMPLOYED
0,100002,1.0,0,0,0,202500.0,406597.5,24700.5,351000.0,0,...,0.374512,0.374512,2.150391,0.468018,1.175716,1.175716,0.064026,0.092407,0.951334,1.372057
1,100003,0.0,0,1,0,270000.0,1293502.5,35698.5,1129500.0,1,...,1.583984,0.186646,15.710938,1.813477,0.290662,0.239484,0.077820,0.083557,1.098485,1.179082
2,100004,0.0,1,0,1,67500.0,135000.0,6750.0,135000.0,0,...,0.793457,0.793457,1.566406,1.051758,0.187652,0.187652,0.042786,0.045532,3.622222,3.853333
3,100006,0.0,0,1,0,135000.0,312682.5,29686.5,297000.0,0,...,0.735840,0.083618,23.296875,2.121094,2.643133,1.497703,0.018188,NaN,0.113722,NaN
4,100007,0.0,0,0,0,121500.0,513000.0,21865.5,513000.0,0,...,0.561523,0.000008,1.037109,0.558594,0.931669,0.538926,0.061340,0.057648,0.402513,0.378209


In [8]:
lgbm_categorical_feat = [
            'CODE_GENDER', 'FLAG_OWN_CAR', 'NAME_CONTRACT_TYPE', 'NAME_EDUCATION_TYPE',
            'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE',
            'ORGANIZATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_TYPE_SUITE', 'WALLSMATERIAL_MODE']

In [9]:
# general cleaning procedures
df = df[df['CODE_GENDER'] != 'XNA']
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

### CODE_GENDER, 
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

### NAME_INCOME_TYPE
df['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
df['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
### NAME_FAMILY_STATUS
df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
df['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)

### 
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,CURRENT_TO_APPROVED_ANNUITY_MEAN_RATIO,PAYMENT_MIN_TO_ANNUITY_RATIO,PAYMENT_MAX_TO_ANNUITY_RATIO,PAYMENT_MEAN_TO_ANNUITY_RATIO,CTA_CREDIT_TO_ANNUITY_MAX_RATIO,CTA_CREDIT_TO_ANNUITY_MEAN_RATIO,DAYS_DECISION_MEAN_TO_BIRTH,DAYS_CREDIT_MEAN_TO_BIRTH,DAYS_DECISION_MEAN_TO_EMPLOYED,DAYS_CREDIT_MEAN_TO_EMPLOYED
0,100002,1.0,0,0,0,202500.0,406597.5,24700.5,351000.0,0,...,0.374512,0.374512,2.150391,0.468018,1.175716,1.175716,0.064026,0.092407,0.951334,1.372057
1,100003,0.0,0,1,0,270000.0,1293502.5,35698.5,1129500.0,1,...,1.583984,0.186646,15.710938,1.813477,0.290662,0.239484,0.077820,0.083557,1.098485,1.179082
2,100004,0.0,1,0,1,67500.0,135000.0,6750.0,135000.0,0,...,0.793457,0.793457,1.566406,1.051758,0.187652,0.187652,0.042786,0.045532,3.622222,3.853333
3,100006,0.0,0,1,0,135000.0,312682.5,29686.5,297000.0,0,...,0.735840,0.083618,23.296875,2.121094,2.643133,1.497703,0.018188,NaN,0.113722,NaN
4,100007,0.0,0,0,0,121500.0,513000.0,21865.5,513000.0,0,...,0.561523,0.000008,1.037109,0.558594,0.931669,0.538926,0.061340,0.057648,0.402513,0.378209


In [10]:
#### one-hotencoding実行
for col in lgbm_categorical_feat:
    df = pd.get_dummies(df, columns=[col], dummy_na=True)

### 列名の不正な文字を置換(ドット対応)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

### 極大値、極小値をNULLへ置換
df = df.replace([np.inf, -np.inf], np.nan)

df.head()

,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_TYPE_SUITE_nan,WALLSMATERIAL_MODE__1_0,WALLSMATERIAL_MODE_0_0,WALLSMATERIAL_MODE_1_0,WALLSMATERIAL_MODE_2_0,WALLSMATERIAL_MODE_3_0,WALLSMATERIAL_MODE_4_0,WALLSMATERIAL_MODE_5_0,WALLSMATERIAL_MODE_6_0,WALLSMATERIAL_MODE_nan
0,100002,1.0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,0,0,1,0,0,0,0,0,0,0
1,100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,0,0,0,1,0,0,0,0,0,0
2,100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,0,1,0,0,0,0,0,0,0,0
3,100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,0,1,0,0,0,0,0,0,0,0
4,100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,0,1,0,0,0,0,0,0,0,0


In [11]:
### 変数選択01
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 793 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 781 features are remained after removing non-informative features
78 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 703 features are remained after removing features not interesting for LightGBM classifier
---=> 22 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 725 features

---=> df final shape: (356250, 725)  <=--- 



,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,DOCUMENT_COUNT_high_risk,DOCUMENT_COUNT_low_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_high_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk
0,100002,1.0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,1,0,1,0,1,0,0,0,1,0
1,100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,1,0,1,0,0,1,0,0,0,1
2,100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,0,1,0,1,0,1,0,0,0,1
3,100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,1,0,1,0,0,1,1,0,0,1
4,100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,1,0,1,0,0,1,1,0,0,1


In [12]:
### 変数選択02
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 725 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 724 features are remained after removing non-informative features
19 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 705 features are remained after removing features not interesting for LightGBM classifier
---=> 9 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 714 features

---=> df final shape: (356250, 714)  <=--- 



,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,POS_COMPLETED_BEFORE_MEAN_low_risk,REGION_RATING_CLIENT_W_CITY_low_risk,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk
0,100002,1.0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,0,1,0,1,0,1,0,0,0,0
1,100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,1,1,0,1,0,1,0,1,0,0
2,100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,1,1,0,1,0,0,1,1,0,0
3,100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,1,1,0,0,0,1,0,1,1,0
4,100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,1,1,0,1,0,1,0,1,1,0


In [13]:
df_train, df_test = FeatureEngineering3.get_apps_all_train_test(df)
df_train.head()

,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,POS_COMPLETED_BEFORE_MEAN_low_risk,REGION_RATING_CLIENT_W_CITY_low_risk,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk
0,100002,1.0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,0,1,0,1,0,1,0,0,0,0
1,100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,1,1,0,1,0,1,0,1,0,0
2,100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,1,1,0,1,0,0,1,1,0,0
3,100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,1,1,0,0,0,1,0,1,1,0
4,100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,1,1,0,1,0,1,0,1,1,0


In [14]:
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,POS_COMPLETED_BEFORE_MEAN_low_risk,REGION_RATING_CLIENT_W_CITY_low_risk,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk
307506,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,1,0,1,0,1,0,1,0,0
307507,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,1,1,0,1,0,1,0,1,0,0
307508,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,1,1,0,1,0,1,0,1,0,0
307509,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,1,1,0,1,0,1,0,0,1,0
307510,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,1,0,0,0,1,0,0,0,0


In [15]:
df_test

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,POS_COMPLETED_BEFORE_MEAN_low_risk,REGION_RATING_CLIENT_W_CITY_low_risk,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk
307506,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,1,0,1,0,1,0,1,0,0
307507,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,1,1,0,1,0,1,0,1,0,0
307508,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,1,1,0,1,0,1,0,1,0,0
307509,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,1,1,0,1,0,1,0,0,1,0
307510,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356245,456221,121500.0,412560.0,17473.5,270000.0,0.002043,-19970,-5168.0,-9096.0,-3399,...,1,0,0,1,0,1,0,1,0,0
356246,456222,157500.0,622413.0,31909.5,495000.0,0.035797,-11186,-1149.0,-3016.0,-3003,...,0,1,0,0,0,1,0,0,0,0
356247,456223,202500.0,315000.0,33205.5,315000.0,0.026398,-15922,-3036.0,-2680.0,-1504,...,1,1,0,1,0,1,0,1,1,0
356248,456224,225000.0,450000.0,25128.0,450000.0,0.018845,-13968,-2732.0,-1461.0,-1364,...,0,1,0,1,0,1,0,0,1,0


In [16]:
train_setB1a = pd.read_csv('./home-credit-default-risk/exports/train_setB1a.csv')
test_setB1a =  pd.read_csv('./home-credit-default-risk/exports/test_setB1a.csv')
test_setB1a.head()

,SK_ID_CURR,setB1a
0,100001,0.033433
1,100005,0.098996
2,100013,0.042185
3,100028,0.039429
4,100038,0.161166


In [17]:
df_train = df_train.merge(train_setB1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_setB1a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,REGION_RATING_CLIENT_W_CITY_low_risk,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,1,0,1,0,1,0,1,0,0,0.033433
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,1,0,1,0,1,0,1,0,0,0.098996
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,1,0,1,0,1,0,1,0,0,0.042185
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,1,0,1,0,1,0,0,1,0,0.039429
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,1,0,0,0,1,0,0,0,0,0.161166


In [18]:
train_setB1xgb = pd.read_csv('./home-credit-default-risk/exports/train_setB1xgb.csv')
test_setB1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_setB1xgb.csv')
test_setB1xgb.head()

,SK_ID_CURR,setB1x
0,100001,0.032762
1,100005,0.100620
2,100013,0.044999
3,100028,0.034211
4,100038,0.172532


In [19]:
df_train = df_train.merge(train_setB1xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_setB1xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,1,0,1,0,1,0,0,0.033433,0.032762
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0,1,0,1,0,1,0,0,0.098996,0.100620
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0,1,0,1,0,1,0,0,0.042185,0.044999
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0,1,0,1,0,0,1,0,0.039429,0.034211
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,0,0,1,0,0,0,0,0.161166,0.172532


In [20]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.040068
1,100005,0.095240
2,100013,0.044435
3,100028,0.036586
4,100038,0.141754


In [21]:
df_train = df_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,1,0,1,0,1,0,0,0.033433,0.032762,0.040068
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,1,0,1,0,1,0,0,0.098996,0.100620,0.095240
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,1,0,1,0,1,0,0,0.042185,0.044999,0.044435
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,1,0,1,0,0,1,0,0.039429,0.034211,0.036586
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,0,1,0,0,0,0,0.161166,0.172532,0.141754


In [22]:
train_set1xgb = pd.read_csv('./home-credit-default-risk/exports/train_set1xgb.csv')
test_set1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set1xgb.csv')
test_set1xgb.head()

,SK_ID_CURR,set1x
0,100001,0.035234
1,100005,0.107426
2,100013,0.048029
3,100028,0.032975
4,100038,0.161962


In [23]:
df_train = df_train.merge(train_set1xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set1xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,AMT_REQ_CREDIT_BUREAU_WEEK_high_risk,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1,set1x
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,1,0,1,0,0,0.033433,0.032762,0.040068,0.035234
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0,1,0,1,0,0,0.098996,0.100620,0.095240,0.107426
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0,1,0,1,0,0,0.042185,0.044999,0.044435,0.048029
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0,1,0,0,1,0,0.039429,0.034211,0.036586,0.032975
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,1,0,0,0,0,0.161166,0.172532,0.141754,0.161962


In [24]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.042769
1,100005,0.103694
2,100013,0.036006
3,100028,0.048812
4,100038,0.173045


In [25]:
df_train = df_train.merge(train_set2a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,NEW_DOC_KURT_high_risk,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1,set1x,set2a
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,1,0,1,0,0,0.033433,0.032762,0.040068,0.035234,0.042769
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,1,0,1,0,0,0.098996,0.100620,0.095240,0.107426,0.103694
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,1,0,1,0,0,0.042185,0.044999,0.044435,0.048029,0.036006
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,1,0,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,1,0,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045


In [26]:
train_set2xgb = pd.read_csv('./home-credit-default-risk/exports/train_set2xgb.csv')
test_set2xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set2xgb.csv')
test_set2xgb.head()

,SK_ID_CURR,set2x
0,100001,0.035013
1,100005,0.103532
2,100013,0.048415
3,100028,0.032249
4,100038,0.150442


In [27]:
df_train = df_train.merge(train_set2xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,NEW_DOC_KURT_low_risk,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,1,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0,1,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0,1,0,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442


In [28]:
train_set2nn = pd.read_csv('./home-credit-default-risk/exports/train_set2nn.csv')
test_set2nn =  pd.read_csv('./home-credit-default-risk/exports/test_set2nn.csv')
test_set2nn.head()

,SK_ID_CURR,set2nn
0,100001,0.048972
1,100005,0.141351
2,100013,0.028088
3,100028,0.039056
4,100038,0.192637


In [29]:
df_train = df_train.merge(train_set2nn, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2nn, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,AGE_RANGE_low_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,1,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,1,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,1,0,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637


In [30]:
train_set3xgb = pd.read_csv('./home-credit-default-risk/exports/train_set3xgb.csv')
test_set3xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set3xgb.csv')
test_set3xgb.head()

,SK_ID_CURR,set3x
0,100001,0.034310
1,100005,0.127556
2,100013,0.044872
3,100028,0.030915
4,100038,0.201295


In [31]:
df_train = df_train.merge(train_set3xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set3xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_high_risk,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.034310
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.127556
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.044872
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.030915
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.201295


In [32]:
train_set4a = pd.read_csv('./home-credit-default-risk/exports/train_set4a.csv')
test_set4a =  pd.read_csv('./home-credit-default-risk/exports/test_set4a.csv')
test_set4a.head()

,SK_ID_CURR,set4a
0,100001,0.042113
1,100005,0.132192
2,100013,0.057271
3,100028,0.042206
4,100038,0.195048


In [33]:
df_train = df_train.merge(train_set4a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set4a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,PREV_ACTIVE_SK_ID_PREV_NUNIQUE_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.034310,0.042113
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.127556,0.132192
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.044872,0.057271
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.030915,0.042206
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.201295,0.195048


In [34]:
train_set4xgb = pd.read_csv('./home-credit-default-risk/exports/train_set4xgb.csv')
test_set4xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set4xgb.csv')
test_set4xgb.head()

,SK_ID_CURR,set4x
0,100001,0.034978
1,100005,0.118268
2,100013,0.039774
3,100028,0.037283
4,100038,0.164890


In [35]:
df_train = df_train.merge(train_set4xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set4xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.034310,0.042113,0.034978
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.127556,0.132192,0.118268
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.044872,0.057271,0.039774
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.030915,0.042206,0.037283
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.201295,0.195048,0.164890


In [36]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
0,100002,1.0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,0.283935,0.234856,0.260928,0.199398,0.242499,0.198794,0.240473,0.259405,0.218256,0.207998
1,100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,0.023060,0.023065,0.012171,0.012061,0.017857,0.012648,0.014736,0.023080,0.012283,0.013902
2,100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,0.032398,0.038719,0.039950,0.047440,0.033203,0.033060,0.029793,0.033290,0.033160,0.034973
3,100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,0.028579,0.034008,0.030776,0.030071,0.048900,0.025442,0.031804,0.022860,0.029616,0.040586
4,100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,0.023252,0.027813,0.035088,0.023555,0.043556,0.039186,0.060946,0.033131,0.035875,0.036123


In [37]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
0,100001,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,-812,...,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.034310,0.042113,0.034978
1,100005,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,-1623,...,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.127556,0.132192,0.118268
2,100013,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,-3503,...,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.044872,0.057271,0.039774
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,-4208,...,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.030915,0.042206,0.037283
4,100038,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,-4262,...,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.201295,0.195048,0.164890


In [38]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')

In [39]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,-2120,...,0.283935,0.234856,0.260928,0.199398,0.242499,0.198794,0.240473,0.259405,0.218256,0.207998
100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,-291,...,0.023060,0.023065,0.012171,0.012061,0.017857,0.012648,0.014736,0.023080,0.012283,0.013902
100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,-2531,...,0.032398,0.038719,0.039950,0.047440,0.033203,0.033060,0.029793,0.033290,0.033160,0.034973
100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,-2437,...,0.028579,0.034008,0.030776,0.030071,0.048900,0.025442,0.031804,0.022860,0.029616,0.040586
100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,-3458,...,0.023252,0.027813,0.035088,0.023555,0.043556,0.039186,0.060946,0.033131,0.035875,0.036123


In [40]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,0.379883,0.579149,...,2.106458,1.635142,1.896448,1.301762,1.758129,1.262144,1.746196,1.888223,1.449139,1.374899
100003,0.0,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,1.079102,1.790846,...,-0.598379,-0.587670,-0.711865,-0.698289,-0.672096,-0.692681,-0.686084,-0.587171,-0.715210,-0.685605
100004,0.0,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,0.206055,0.306866,...,-0.501560,-0.423376,-0.420592,-0.320577,-0.506078,-0.478319,-0.523846,-0.480227,-0.495838,-0.461924
100006,0.0,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,-1.375000,0.369140,...,-0.541154,-0.472816,-0.516790,-0.506012,-0.336265,-0.558316,-0.502178,-0.589469,-0.533075,-0.402327
100007,0.0,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,0.191284,-0.307264,...,-0.596392,-0.537838,-0.471570,-0.575581,-0.394081,-0.413984,-0.188179,-0.481891,-0.467308,-0.449710


In [41]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
0,100002,1.0,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,0.379883,...,2.106458,1.635142,1.896448,1.301762,1.758129,1.262144,1.746196,1.888223,1.449139,1.374899
1,100003,0.0,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,1.079102,...,-0.598379,-0.587670,-0.711865,-0.698289,-0.672096,-0.692681,-0.686084,-0.587171,-0.715210,-0.685605
2,100004,0.0,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,0.206055,...,-0.501560,-0.423376,-0.420592,-0.320577,-0.506078,-0.478319,-0.523846,-0.480227,-0.495838,-0.461924
3,100006,0.0,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,-1.375000,...,-0.541154,-0.472816,-0.516790,-0.506012,-0.336265,-0.558316,-0.502178,-0.589469,-0.533075,-0.402327
4,100007,0.0,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,0.191284,...,-0.596392,-0.537838,-0.471570,-0.575581,-0.394081,-0.413984,-0.188179,-0.481891,-0.467308,-0.449710


In [42]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 764.82 MB
Memory usage of optimization is 890537504.00 MB
Decreased by -116436865.0%
Memory usage of dataframe is 121.14 MB
Memory usage of optimization is 140967776.00 MB
Decreased by -116365268.0%


,SK_ID_CURR,TARGET,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3x,set4a,set4x
0,100002,1.0,0.313232,-0.478027,-0.166138,-0.507324,-0.149780,1.506836,0.747070,0.379883,...,2.107422,1.634766,1.896484,1.301758,1.757812,1.261719,1.746094,1.888672,1.449219,1.375000
1,100003,0.0,0.933594,1.725586,0.592773,1.599609,-1.252930,-0.166870,0.511719,1.079102,...,-0.598145,-0.587891,-0.711914,-0.698242,-0.671875,-0.692871,-0.686035,-0.587402,-0.715332,-0.685547
2,100004,0.0,-0.927246,-1.153320,-1.404297,-1.091797,-0.783203,-0.689453,0.923828,0.206055,...,-0.501465,-0.423340,-0.420654,-0.320557,-0.505859,-0.478271,-0.523926,-0.480225,-0.495850,-0.461914
3,100006,0.0,-0.307129,-0.711426,0.177856,-0.653320,-0.928711,-0.680176,-0.280518,-1.375000,...,-0.541016,-0.472900,-0.516602,-0.505859,-0.336182,-0.558105,-0.501953,-0.589355,-0.533203,-0.402344
4,100007,0.0,-0.431152,-0.213745,-0.361816,-0.068726,0.562988,-0.892578,-0.279785,0.191284,...,-0.596191,-0.537598,-0.471680,-0.575684,-0.394043,-0.414062,-0.188232,-0.481934,-0.467285,-0.449707


In [43]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [44]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [45]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 722) (61502, 722)
[100]	training's auc: 0.801231	valid_1's auc: 0.791815
[200]	training's auc: 0.805323	valid_1's auc: 0.793133
[300]	training's auc: 0.809375	valid_1's auc: 0.793896
[400]	training's auc: 0.813305	valid_1's auc: 0.794224
[500]	training's auc: 0.81706	valid_1's auc: 0.794343
[600]	training's auc: 0.820641	valid_1's auc: 0.79464
[700]	training's auc: 0.824001	valid_1's auc: 0.794672
[800]	training's auc: 0.827376	valid_1's auc: 0.794661
[900]	training's auc: 0.830635	valid_1's auc: 0.794657
[1000]	training's auc: 0.833665	valid_1's auc: 0.794758
[1100]	training's auc: 0.836631	valid_1's auc: 0.794687
[1200]	training's auc: 0.839421	valid_1's auc: 0.794667
[1300]	training's auc: 0.842192	valid_1's auc: 0.794663
[1400]	training's auc: 0.844991	valid_1's auc: 0.794706
[1500]	training's auc: 0.847686	valid_1's auc: 0.794691
[1600]	training's auc: 0.850402	valid_1's auc: 0.794681
[1700]	training's auc: 0.852971	valid_1's au

In [46]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
716,set2x,609.8,15.498387
719,set4x,548.2,12.049896
300,EXT_SOURCE_3,425.6,29.194178
130,BUREAU_CONSUMER_DAYS_CREDIT_ENDDATE_MAX,409.4,28.762823
316,GROUP_INCOME_MEAN,406.4,22.244100
714,set2a,390.0,15.280707
721,setB1x,349.4,25.085853
11,AMT_ANNUITY,345.6,21.995454
319,INCOME_TO_BIRTH_RATIO,333.2,27.298352
576,PREV_Consumer_AMT_ANNUITY_MEAN,331.2,19.123284


In [47]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [48]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [49]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_OS_5-1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.036326
1,100005,0.110277
2,100013,0.039769
3,100028,0.044291
4,100038,0.143220
